In [1]:
import numpy as np 
import time

import torch 
import torch.nn as nn
import torch.nn.functional as F 

import matplotlib.pyplot as plt
from collections import OrderedDict

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.preprocessing import RobustScaler

from utils.inference import Trainer, plot_loss
from utils.models import DNN
import utils.datasets as d

In [2]:
from utils.inception import Inception, InceptionBlock, correct_sizes
from utils import layers

In [3]:
X = np.vstack((np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt1.npy"),
               np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt2.npy")))
y = np.load("data/sequenced_data_for_VAE_length-160_stride-10_targets.npy")

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666)

In [5]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=666)

In [7]:
train = d.H_alphaSequences(X_train, y_train)
valid = d.H_alphaSequences(X_validation, y_validation)
test = d.H_alphaSequences(X_test, y_test)

In [8]:
correct_sizes([6,12,24])

[5, 11, 23]

In [79]:
IT = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            InceptionBlock(
                in_channels=1, 
                n_filters=32, 
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                use_residual=True,
                activation=nn.ReLU()
            ),
            nn.AdaptiveAvgPool1d(output_size=1),
            layers.Flatten(out_features=32*4*1),
            nn.Linear(in_features=4*32*1, out_features=4)
)

In [80]:
IT

Sequential(
  (0): Reshape()
  (1): InceptionBlock(
    (activation): ReLU()
    (inception_1): Inception(
      (conv_from_bottleneck_1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv_from_bottleneck_2): Conv1d(1, 32, kernel_size=(11,), stride=(1,), padding=(5,), bias=False)
      (conv_from_bottleneck_3): Conv1d(1, 32, kernel_size=(23,), stride=(1,), padding=(11,), bias=False)
      (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (conv_from_maxpool): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
      (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
    (inception_2): Inception(
      (bottleneck): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
      (conv_from_bottleneck_1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv_from_bottleneck_2): Conv1d(32, 32, kernel_siz

In [81]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [82]:
optimizer= torch.optim.Adam(IT.parameters(), lr=3e-3)

In [83]:
m1 = Trainer(
        model=IT,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,20], gamma=0.5),
        tensorboard=True,
        model_name="InceptionTime_lr-3e-3_bs-512_ks-[5,11,23]",
        verbose=True
        )

cuda:0


In [84]:
lh = m1(epochs=range(50), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/50], average_loss:0.6614, validation_loss:0.5230, val_accuracy:0.7403
Epoch [2/50], average_loss:0.4672, validation_loss:0.3994, val_accuracy:0.8701
Epoch [3/50], average_loss:0.3930, validation_loss:0.4434, val_accuracy:0.8701
Epoch [4/50], average_loss:0.3608, validation_loss:0.4828, val_accuracy:0.8442
Epoch [5/50], average_loss:0.3407, validation_loss:0.3702, val_accuracy:0.8831
Epoch [6/50], average_loss:0.3237, validation_loss:0.3469, val_accuracy:0.7532
Epoch [7/50], average_loss:0.3177, validation_loss:0.2992, val_accuracy:0.9091
Epoch [8/50], average_loss:0.3101, validation_loss:0.7815, val_accuracy:0.7922
Epoch [9/50], average_loss:0.3058, validation_loss:0.3221, val_accuracy:0.8961
Epoch [10/50], average_loss:0.2949, validation_loss:0.3329, val_accuracy:0.8961
Epoch [11/50], average_loss:0.2908, validation_loss:0.4218, val_accuracy:0.8961
Epoch [12/50], average_loss:0.2844, validation_loss:0.3169, val_accuracy:0.8052
Epoch [13/50], average_loss:0.2776, validation_lo

In [50]:
m1.tb.close()

In [27]:
X_test.shape

(16736, 160)

In [88]:
#torch.save(m1.model, "models_and_losses/InceptionTime_lr-3e-3_bs-512_ks-[5,11,23]_51-epochs.pt")

In [120]:
m1.model.eval()
with torch.no_grad():
    x_pred = np.argmax(m1.model(torch.tensor(X_test).float()).detach(), axis=1)

In [121]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [122]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8218793419973198

In [123]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9038001912045889

In [124]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

array([[ 3244,   398,    25,   568],
       [  132, 10218,    18,    77],
       [   21,    68,   335,    55],
       [   92,   118,    38,  1329]], dtype=int64)

In [111]:
def sum_params(model):
    s = []
    for p in model.parameters():
        dims = p.size()
        n = np.prod(p.size())
        s.append((dims, n))
    return s, np.sum([j for i,j in s])

In [112]:
sum_params(m1.model)

([(torch.Size([32, 1, 5]), 160),
  (torch.Size([32, 1, 11]), 352),
  (torch.Size([32, 1, 23]), 736),
  (torch.Size([32, 1, 1]), 32),
  (torch.Size([128]), 128),
  (torch.Size([128]), 128),
  (torch.Size([32, 128, 1]), 4096),
  (torch.Size([32, 32, 5]), 5120),
  (torch.Size([32, 32, 11]), 11264),
  (torch.Size([32, 32, 23]), 23552),
  (torch.Size([32, 128, 1]), 4096),
  (torch.Size([128]), 128),
  (torch.Size([128]), 128),
  (torch.Size([32, 128, 1]), 4096),
  (torch.Size([32, 32, 5]), 5120),
  (torch.Size([32, 32, 11]), 11264),
  (torch.Size([32, 32, 23]), 23552),
  (torch.Size([32, 128, 1]), 4096),
  (torch.Size([128]), 128),
  (torch.Size([128]), 128),
  (torch.Size([128, 1, 1]), 128),
  (torch.Size([128]), 128),
  (torch.Size([128]), 128),
  (torch.Size([128]), 128),
  (torch.Size([4, 128]), 512),
  (torch.Size([4]), 4)],
 99332)

# lr scheadule

In [9]:
IT1 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            InceptionBlock(
                in_channels=1, 
                n_filters=32, 
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                use_residual=True,
                activation=nn.ReLU()
            ),
            nn.AdaptiveAvgPool1d(output_size=1),
            layers.Flatten(out_features=32*4*1),
            nn.Linear(in_features=4*32*1, out_features=4)
)

In [10]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [11]:
optimizer= torch.optim.Adam(IT1.parameters(), lr=5e-3)

In [12]:
m2 = Trainer(
        model=IT1,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,20], gamma=0.2),
        tensorboard=True,
        model_name="InceptionTime_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]",
        verbose=True
        )

cuda:0


In [13]:
lh = m2(epochs=range(50), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/50], average_loss:0.6230, validation_loss:0.5540, val_accuracy:0.7273
Epoch [2/50], average_loss:0.4504, validation_loss:0.6683, val_accuracy:0.7403
Epoch [3/50], average_loss:0.3808, validation_loss:0.4038, val_accuracy:0.8961
Epoch [4/50], average_loss:0.3589, validation_loss:0.7224, val_accuracy:0.7662
Epoch [5/50], average_loss:0.3503, validation_loss:0.3807, val_accuracy:0.8312
Epoch [6/50], average_loss:0.3077, validation_loss:0.3043, val_accuracy:0.8961
Epoch [7/50], average_loss:0.2977, validation_loss:0.2923, val_accuracy:0.9091
Epoch [8/50], average_loss:0.2949, validation_loss:0.3146, val_accuracy:0.9351
Epoch [9/50], average_loss:0.2893, validation_loss:0.2843, val_accuracy:0.8961
Epoch [10/50], average_loss:0.2870, validation_loss:0.2858, val_accuracy:0.9221
Epoch [11/50], average_loss:0.2855, validation_loss:0.2904, val_accuracy:0.8182
Epoch [12/50], average_loss:0.2800, validation_loss:0.2747, val_accuracy:0.9091
Epoch [13/50], average_loss:0.2819, validation_lo

In [24]:
lh = m2(epochs=range(50,100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [51/100], average_loss:0.2261, validation_loss:0.2218, val_accuracy:0.9221
Epoch [52/100], average_loss:0.2258, validation_loss:0.2261, val_accuracy:0.9351
Epoch [53/100], average_loss:0.2244, validation_loss:0.2264, val_accuracy:0.9351
Epoch [54/100], average_loss:0.2250, validation_loss:0.2347, val_accuracy:0.9091
Epoch [55/100], average_loss:0.2250, validation_loss:0.2251, val_accuracy:0.9091
Epoch [56/100], average_loss:0.2237, validation_loss:0.2446, val_accuracy:0.9221
Epoch [57/100], average_loss:0.2247, validation_loss:0.2216, val_accuracy:0.9351
Epoch [58/100], average_loss:0.2234, validation_loss:0.3219, val_accuracy:0.8442
Epoch [59/100], average_loss:0.2235, validation_loss:0.2198, val_accuracy:0.8831
Epoch [60/100], average_loss:0.2204, validation_loss:0.2335, val_accuracy:0.9610
Epoch [61/100], average_loss:0.2211, validation_loss:0.2195, val_accuracy:0.9091
Epoch [62/100], average_loss:0.2223, validation_loss:0.2251, val_accuracy:0.9091
Epoch [63/100], average_loss

In [25]:
m2.model.eval()
m2.model.cpu()
with torch.no_grad():
    x_pred = np.argmax(m2.model(torch.tensor(X_test).float()).detach(), axis=1)
m2.model.cuda()

Sequential(
  (0): Reshape()
  (1): InceptionBlock(
    (activation): ReLU()
    (inception_1): Inception(
      (conv_from_bottleneck_1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv_from_bottleneck_2): Conv1d(1, 32, kernel_size=(11,), stride=(1,), padding=(5,), bias=False)
      (conv_from_bottleneck_3): Conv1d(1, 32, kernel_size=(23,), stride=(1,), padding=(11,), bias=False)
      (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (conv_from_maxpool): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
      (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
    (inception_2): Inception(
      (bottleneck): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
      (conv_from_bottleneck_1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv_from_bottleneck_2): Conv1d(32, 32, kernel_siz

In [17]:
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [18]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8401764734820084

In [19]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9152724665391969

In [20]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

array([[ 3619,   455,    24,   137],
       [  191, 10142,    15,    97],
       [   34,    64,   320,    61],
       [  205,   102,    33,  1237]], dtype=int64)

In [22]:
#torch.save(m2.model, "models_and_losses/InceptionTime_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]_50-epochs.pt")

In [26]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [27]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8658205936831611

In [28]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9246534416826003

In [29]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

array([[ 3858,   256,    43,    78],
       [  297, 10026,    32,    90],
       [   19,    28,   407,    25],
       [  280,    70,    43,  1184]], dtype=int64)

In [30]:
#torch.save(m2.model, "models_and_losses/InceptionTime_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]_100-epochs.pt")

# 4 Layer inceptionTime

In [8]:
IT2 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            InceptionBlock(
                in_channels=1, 
                n_filters=32, 
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                use_residual=True,
                activation=nn.LeakyReLU()
            ),
            Inception(
                in_channels=32*4,
                n_filters=32,
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                activation=nn.LeakyReLU()
            ),
            nn.AdaptiveAvgPool1d(output_size=1),
            layers.Flatten(out_features=32*4*1),
            nn.Linear(in_features=4*32*1, out_features=4)
)

In [9]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [10]:
optimizer= torch.optim.Adam(IT2.parameters(), lr=5e-3)

In [11]:
m3 = Trainer(
        model=IT2,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        #scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,20], gamma=0.5),
        tensorboard=True,
        model_name="InceptionTime4_lr-5e-3_bs-512_ks-[5,11,23]",
        verbose=True
        )

cuda:0


In [12]:
lh = m3(epochs=range(50), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/50], average_loss:0.6025, validation_loss:0.8079, val_accuracy:0.5844
Epoch [2/50], average_loss:0.4319, validation_loss:1.2168, val_accuracy:0.6883
Epoch [3/50], average_loss:0.3627, validation_loss:1.1021, val_accuracy:0.6234
Epoch [4/50], average_loss:0.3451, validation_loss:1.0093, val_accuracy:0.7532
Epoch [5/50], average_loss:0.3148, validation_loss:0.3879, val_accuracy:0.8182
Epoch [6/50], average_loss:0.3119, validation_loss:0.3937, val_accuracy:0.8442
Epoch [7/50], average_loss:0.2931, validation_loss:0.4985, val_accuracy:0.7792
Epoch [8/50], average_loss:0.2823, validation_loss:0.3607, val_accuracy:0.8571
Epoch [9/50], average_loss:0.2712, validation_loss:0.3288, val_accuracy:0.9091
Epoch [10/50], average_loss:0.2568, validation_loss:0.2932, val_accuracy:0.9221
Epoch [11/50], average_loss:0.2518, validation_loss:0.3663, val_accuracy:0.8052
Epoch [12/50], average_loss:0.2462, validation_loss:0.3236, val_accuracy:0.8701
Epoch [13/50], average_loss:0.2410, validation_lo

In [13]:
m3.model.eval()
m3.model.cpu()
with torch.no_grad():
    x_pred = np.argmax(m3.model(torch.tensor(X_test).float()).detach(), axis=1)
m3.model.cuda()

Sequential(
  (0): Reshape()
  (1): InceptionBlock(
    (activation): LeakyReLU(negative_slope=0.01)
    (inception_1): Inception(
      (conv_from_bottleneck_1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv_from_bottleneck_2): Conv1d(1, 32, kernel_size=(11,), stride=(1,), padding=(5,), bias=False)
      (conv_from_bottleneck_3): Conv1d(1, 32, kernel_size=(23,), stride=(1,), padding=(11,), bias=False)
      (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (conv_from_maxpool): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
      (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.01)
    )
    (inception_2): Inception(
      (bottleneck): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
      (conv_from_bottleneck_1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (co

In [14]:
x_pred

tensor([1, 1, 1,  ..., 1, 1, 0])

In [15]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8530326151058839

In [16]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9020673996175909

In [ ]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

In [18]:
m3.tb.close()

# lr scheaduler

In [8]:
IT2 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            InceptionBlock(
                in_channels=1, 
                n_filters=32, 
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                use_residual=True,
                activation=nn.LeakyReLU()
            ),
            Inception(
                in_channels=32*4,
                n_filters=32,
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                activation=nn.LeakyReLU()
            ),
            nn.AdaptiveAvgPool1d(output_size=1),
            layers.Flatten(out_features=32*4*1),
            nn.Linear(in_features=4*32*1, out_features=4)
)

In [9]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [10]:
optimizer= torch.optim.Adam(IT2.parameters(), lr=5e-3)

In [11]:
m4 = Trainer(
        model=IT2,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,30], gamma=0.5),
        tensorboard=True,
        model_name="InceptionTime4_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]",
        verbose=True
        )

cuda:0


In [12]:
lh = m4(epochs=range(50), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/50], average_loss:0.6404, validation_loss:0.5349, val_accuracy:0.7403
Epoch [2/50], average_loss:0.4506, validation_loss:2.0383, val_accuracy:0.5455
Epoch [3/50], average_loss:0.4292, validation_loss:0.6249, val_accuracy:0.8052
Epoch [4/50], average_loss:0.3623, validation_loss:0.8582, val_accuracy:0.7532
Epoch [5/50], average_loss:0.3298, validation_loss:0.3587, val_accuracy:0.8701
Epoch [6/50], average_loss:0.3180, validation_loss:0.5049, val_accuracy:0.8312
Epoch [7/50], average_loss:0.2982, validation_loss:0.3569, val_accuracy:0.8701
Epoch [8/50], average_loss:0.2888, validation_loss:0.2964, val_accuracy:0.9351
Epoch [9/50], average_loss:0.2883, validation_loss:0.5106, val_accuracy:0.8442
Epoch [10/50], average_loss:0.2805, validation_loss:0.5030, val_accuracy:0.8442
Epoch [11/50], average_loss:0.2565, validation_loss:0.3158, val_accuracy:0.8571
Epoch [12/50], average_loss:0.2561, validation_loss:0.2618, val_accuracy:0.8831
Epoch [13/50], average_loss:0.2503, validation_lo

In [13]:
m4.model.eval()
m4.model.cpu()
with torch.no_grad():
    x_pred = np.argmax(m4.model(torch.tensor(X_test).float()).detach(), axis=1)
m4.model.cuda()

Sequential(
  (0): Reshape()
  (1): InceptionBlock(
    (activation): LeakyReLU(negative_slope=0.01)
    (inception_1): Inception(
      (conv_from_bottleneck_1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv_from_bottleneck_2): Conv1d(1, 32, kernel_size=(11,), stride=(1,), padding=(5,), bias=False)
      (conv_from_bottleneck_3): Conv1d(1, 32, kernel_size=(23,), stride=(1,), padding=(11,), bias=False)
      (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (conv_from_maxpool): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
      (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.01)
    )
    (inception_2): Inception(
      (bottleneck): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
      (conv_from_bottleneck_1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (co

In [14]:
x_pred

tensor([1, 1, 1,  ..., 1, 1, 1])

In [15]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8483816623000584

In [16]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.8919694072657743

In [17]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred)
cf1

array([[ 3002,  1107,    57,    69],
       [   62, 10264,    39,    80],
       [    9,     8,   447,    15],
       [  203,   112,    47,  1215]], dtype=int64)

In [18]:
m4.tb.close()

# InceptionTime LeakyReLU

In [9]:
IT3 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            InceptionBlock(
                in_channels=1, 
                n_filters=32, 
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                use_residual=True,
                activation=nn.LeakyReLU()
            ),
            nn.AdaptiveAvgPool1d(output_size=1),
            layers.Flatten(out_features=32*4*1),
            nn.Linear(in_features=4*32*1, out_features=4)
)

In [10]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [11]:
optimizer= torch.optim.Adam(IT3.parameters(), lr=5e-3)

In [12]:
m2 = Trainer(
        model=IT3,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,20], gamma=0.2),
        tensorboard=True,
        model_name="InceptionTime_LeakyReLU_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]",
        verbose=True
        )

cuda:0


In [13]:
lh = m2(epochs=range(50), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/50], average_loss:0.6470, validation_loss:0.5115, val_accuracy:0.8312
Epoch [2/50], average_loss:0.4845, validation_loss:0.4740, val_accuracy:0.8571
Epoch [3/50], average_loss:0.4191, validation_loss:0.3817, val_accuracy:0.8182
Epoch [4/50], average_loss:0.3786, validation_loss:0.4483, val_accuracy:0.7792
Epoch [5/50], average_loss:0.3516, validation_loss:0.3518, val_accuracy:0.8961
Epoch [6/50], average_loss:0.3158, validation_loss:0.2989, val_accuracy:0.8831
Epoch [7/50], average_loss:0.3048, validation_loss:0.2955, val_accuracy:0.8442
Epoch [8/50], average_loss:0.3035, validation_loss:0.3288, val_accuracy:0.8571
Epoch [9/50], average_loss:0.2987, validation_loss:0.2913, val_accuracy:0.9221
Epoch [10/50], average_loss:0.2949, validation_loss:0.3034, val_accuracy:0.8831
Epoch [11/50], average_loss:0.2884, validation_loss:0.3012, val_accuracy:0.9091
Epoch [12/50], average_loss:0.2864, validation_loss:0.2882, val_accuracy:0.9351
Epoch [13/50], average_loss:0.2819, validation_lo

In [14]:
m2.model.eval()
m2.model.cpu()
with torch.no_grad():
    x_pred = np.argmax(m2.model(torch.tensor(X_test).float()).detach(), axis=1)
m2.model.cuda()

Sequential(
  (0): Reshape()
  (1): InceptionBlock(
    (activation): LeakyReLU(negative_slope=0.01)
    (inception_1): Inception(
      (conv_from_bottleneck_1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv_from_bottleneck_2): Conv1d(1, 32, kernel_size=(11,), stride=(1,), padding=(5,), bias=False)
      (conv_from_bottleneck_3): Conv1d(1, 32, kernel_size=(23,), stride=(1,), padding=(11,), bias=False)
      (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (conv_from_maxpool): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
      (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.01)
    )
    (inception_2): Inception(
      (bottleneck): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
      (conv_from_bottleneck_1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (co

In [15]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [16]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8370576599423052

In [17]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9152127151051626

In [18]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred) # x_axis = predicted, y_axis = ground_truth
cf1

array([[ 3876,   268,    39,    52],
       [  348, 10006,    30,    61],
       [   40,    41,   371,    27],
       [  325,   112,    76,  1064]], dtype=int64)

In [19]:
#torch.save(m2.model, "models_and_losses/InceptionTime_LeakyReLU_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]_50-epochs.pt" )

C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Reshape. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type InceptionBlock. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ghost_000\Miniconda3\envs\Pytorch\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LeakyReLU. It won't be checked for correc

In [20]:
lh = m2(epochs=range(50,100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [51/100], average_loss:0.2280, validation_loss:0.2316, val_accuracy:0.9221
Epoch [52/100], average_loss:0.2272, validation_loss:0.2383, val_accuracy:0.8571
Epoch [53/100], average_loss:0.2266, validation_loss:0.2255, val_accuracy:0.9221
Epoch [54/100], average_loss:0.2265, validation_loss:0.2542, val_accuracy:0.9221
Epoch [55/100], average_loss:0.2279, validation_loss:0.2243, val_accuracy:0.9481
Epoch [56/100], average_loss:0.2257, validation_loss:0.2465, val_accuracy:0.8701
Epoch [57/100], average_loss:0.2253, validation_loss:0.2264, val_accuracy:0.9351
Epoch [58/100], average_loss:0.2251, validation_loss:0.2344, val_accuracy:0.9091
Epoch [59/100], average_loss:0.2242, validation_loss:0.2395, val_accuracy:0.8961
Epoch [60/100], average_loss:0.2238, validation_loss:0.2377, val_accuracy:0.8571
Epoch [61/100], average_loss:0.2221, validation_loss:0.2218, val_accuracy:0.9740
Epoch [62/100], average_loss:0.2204, validation_loss:0.2348, val_accuracy:0.9091
Epoch [63/100], average_loss

In [21]:
m2.model.eval()
m2.model.cpu()
with torch.no_grad():
    x_pred = np.argmax(m2.model(torch.tensor(X_test).float()).detach(), axis=1)
m2.model.cuda()

Sequential(
  (0): Reshape()
  (1): InceptionBlock(
    (activation): LeakyReLU(negative_slope=0.01)
    (inception_1): Inception(
      (conv_from_bottleneck_1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv_from_bottleneck_2): Conv1d(1, 32, kernel_size=(11,), stride=(1,), padding=(5,), bias=False)
      (conv_from_bottleneck_3): Conv1d(1, 32, kernel_size=(23,), stride=(1,), padding=(11,), bias=False)
      (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (conv_from_maxpool): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
      (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): LeakyReLU(negative_slope=0.01)
    )
    (inception_2): Inception(
      (bottleneck): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
      (conv_from_bottleneck_1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (co

In [22]:
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [23]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.8565825857096815

In [24]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9238169216061185

In [25]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred) # x_axis = predicted, y_axis = ground_truth
cf1

array([[ 3845,   291,    32,    67],
       [  240, 10078,    32,    95],
       [   37,    28,   383,    31],
       [  292,    79,    51,  1155]], dtype=int64)

In [26]:
#torch.save(m2.model, "models_and_losses/InceptionTime_LeakyReLU_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]_100-epochs.pt" )

# InceptionTime - full version

In [9]:
IT4 = nn.Sequential(
            layers.Reshape(out_shape=(1,160)),
            InceptionBlock(
                in_channels=1, 
                n_filters=32, 
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                use_residual=True,
                activation=nn.ReLU()
            ),
            InceptionBlock(
                in_channels=32*4, 
                n_filters=32, 
                kernel_sizes=[5, 11, 23],
                bottleneck_channels=32,
                use_residual=True,
                activation=nn.ReLU()
            ),
            nn.AdaptiveAvgPool1d(output_size=1),
            layers.Flatten(out_features=32*4*1),
            nn.Linear(in_features=4*32*1, out_features=4)
)

In [12]:
IT4

Sequential(
  (0): Reshape()
  (1): InceptionBlock(
    (activation): ReLU()
    (inception_1): Inception(
      (conv_from_bottleneck_1): Conv1d(1, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv_from_bottleneck_2): Conv1d(1, 32, kernel_size=(11,), stride=(1,), padding=(5,), bias=False)
      (conv_from_bottleneck_3): Conv1d(1, 32, kernel_size=(23,), stride=(1,), padding=(11,), bias=False)
      (max_pool): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (conv_from_maxpool): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
      (batch_norm): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
    )
    (inception_2): Inception(
      (bottleneck): Conv1d(128, 32, kernel_size=(1,), stride=(1,), bias=False)
      (conv_from_bottleneck_1): Conv1d(32, 32, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
      (conv_from_bottleneck_2): Conv1d(32, 32, kernel_siz

In [10]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [11]:
optimizer= torch.optim.Adam(IT4.parameters(), lr=5e-3)

In [13]:
m4 = Trainer(
        model=IT4,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,20], gamma=0.2),
        tensorboard=True,
        model_name="InceptionTime_full-version_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]",
        verbose=True
        )

cuda:0


In [14]:
lh = m4(epochs=range(50), train_loader=train_loader, validation_loader=valid_loader)

Epoch [1/50], average_loss:0.5573, validation_loss:2.1409, val_accuracy:0.5584
Epoch [2/50], average_loss:0.4032, validation_loss:0.3330, val_accuracy:0.8701
Epoch [3/50], average_loss:0.3351, validation_loss:0.3136, val_accuracy:0.8831
Epoch [4/50], average_loss:0.3002, validation_loss:0.3735, val_accuracy:0.8571
Epoch [5/50], average_loss:0.2867, validation_loss:0.5746, val_accuracy:0.8312
Epoch [6/50], average_loss:0.2505, validation_loss:0.3357, val_accuracy:0.8961
Epoch [7/50], average_loss:0.2412, validation_loss:0.2728, val_accuracy:0.9091
Epoch [8/50], average_loss:0.2367, validation_loss:0.3128, val_accuracy:0.8571
Epoch [9/50], average_loss:0.2351, validation_loss:0.2752, val_accuracy:0.8961
Epoch [10/50], average_loss:0.2348, validation_loss:0.2698, val_accuracy:0.9351
Epoch [11/50], average_loss:0.2286, validation_loss:0.2236, val_accuracy:0.9610
Epoch [12/50], average_loss:0.2282, validation_loss:0.3389, val_accuracy:0.8312
Epoch [13/50], average_loss:0.2213, validation_lo

In [15]:
m4.model.eval()
m4.model.cpu()
with torch.no_grad():
    x_pred = np.argmax(m4.model(torch.tensor(X_test).float()).detach(), axis=1)
m4.model.cuda()
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [18]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.9003748172987842

In [19]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9422801147227533

In [20]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred) # x_axis = predicted, y_axis = ground_truth
cf1

array([[ 4037,   116,    17,    65],
       [  305, 10102,     7,    31],
       [   26,    23,   408,    22],
       [  262,    75,    17,  1223]], dtype=int64)

In [17]:
#torch.save(m4.model, "models_and_losses/InceptionTime_full-version_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]_50-epochs.pt")

In [15]:
m4.model.eval()
m4.model.cpu()
with torch.no_grad():
    x_pred = np.argmax(m4.model(torch.tensor(X_test).float()).detach(), axis=1)
m4.model.cuda()
x_pred

tensor([1, 1, 1,  ..., 0, 1, 0])

In [18]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.9003748172987842

In [19]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9422801147227533

In [20]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred) # x_axis = predicted, y_axis = ground_truth
cf1

array([[ 4037,   116,    17,    65],
       [  305, 10102,     7,    31],
       [   26,    23,   408,    22],
       [  262,    75,    17,  1223]], dtype=int64)

In [17]:
#torch.save(m4.model, "models_and_losses/InceptionTime_full-version_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]_50-epochs.pt")

In [21]:
lh = m4(epochs=range(50,100), train_loader=train_loader, validation_loader=valid_loader)

Epoch [51/100], average_loss:0.1396, validation_loss:0.1911, val_accuracy:0.9481
Epoch [52/100], average_loss:0.1389, validation_loss:0.1695, val_accuracy:0.9610
Epoch [53/100], average_loss:0.1385, validation_loss:0.2347, val_accuracy:0.9221
Epoch [54/100], average_loss:0.1360, validation_loss:0.1826, val_accuracy:0.9091
Epoch [55/100], average_loss:0.1348, validation_loss:0.1646, val_accuracy:0.9091
Epoch [56/100], average_loss:0.1343, validation_loss:0.2033, val_accuracy:0.9091
Epoch [57/100], average_loss:0.1296, validation_loss:0.1604, val_accuracy:0.8831
Epoch [58/100], average_loss:0.1332, validation_loss:0.1745, val_accuracy:0.8961
Epoch [59/100], average_loss:0.1294, validation_loss:0.2348, val_accuracy:0.8701
Epoch [60/100], average_loss:0.1275, validation_loss:0.1432, val_accuracy:0.9740
Epoch [61/100], average_loss:0.1257, validation_loss:0.2416, val_accuracy:0.9091
Epoch [62/100], average_loss:0.1271, validation_loss:0.1490, val_accuracy:0.9740
Epoch [63/100], average_loss

In [22]:
m4.model.eval()
m4.model.cpu()
with torch.no_grad():
    x_pred = np.argmax(m4.model(torch.tensor(X_test).float()).detach(), axis=1)
m4.model.cuda()
x_pred

tensor([1, 1, 1,  ..., 3, 1, 0])

In [23]:
f1_score(y_true=y_test, y_pred=x_pred,average="macro")

0.9125625555928921

In [24]:
accuracy_score(y_true=y_test, y_pred=x_pred)

0.9493307839388145

In [25]:
cf1 = confusion_matrix(y_true=y_test, y_pred=x_pred) # x_axis = predicted, y_axis = ground_truth
cf1

array([[ 3805,   296,    27,   107],
       [   91, 10307,     8,    39],
       [   22,     8,   425,    24],
       [  169,    37,    20,  1351]], dtype=int64)

In [26]:
#torch.save(m4.model, "models_and_losses/InceptionTime_full-version_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]_100-epochs.pt")

In [29]:
#torch.save(m4.model.state_dict(), "models_and_losses/InceptionTime_full-version_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]_100-epochs_state_dict.pt")

torch.save(the_model.state_dict(), PATH)

the_model = TheModelClass(*args, **kwargs)

the_model.load_state_dict(torch.load(PATH))

In [27]:
best_model = torch.load("models_and_losses/InceptionTime_full-version_lr-{5e-3,1e-3,2e-4},_bs-512_ks-[5,11,23]_100-epochs.pt", map_location=torch.device("cpu"))